In [75]:
import pandas as pd

In [76]:
df = pd.read_csv('data/뇌졸중_환자_수_시도별_월별__20231122122856.csv', encoding='ansi')

In [77]:
df.head()

,지역별(1),2014,2014.1,2014.2,2014.3,2014.4,2014.5,2014.6,2014.7,2014.8,...,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7,2021.8,2021.9,2021.10,2021.11
0,지역별(1),1월,2월,3월,4월,5월,6월,7월,8월,9월,...,3월,4월,5월,6월,7월,8월,9월,10월,11월,12월
1,서울,2012,1871,2003,1970,2037,1882,1849,1900,1907,...,2112,2223,2221,2182,2211,2151,2081,2136,2049,2053
2,부산,391,369,409,362,408,386,380,421,455,...,601,554,645,605,587,550,582,569,565,545
3,대구,673,543,568,607,625,571,566,603,590,...,678,665,716,628,696,610,559,643,691,590
4,인천,341,309,320,293,332,296,299,284,272,...,536,557,617,586,554,567,582,592,586,570


In [78]:
# '지역별(1)' 컬럼에서 '지역별(1)' 문자값을 가진 행을 제거
df = df[df['지역별(1)'] != '지역별(1)']

In [79]:
long_df = df.melt(id_vars=['지역별(1)'], 
                  var_name='STD_YYYYMM', 
                  value_name='뇌졸증')

In [80]:
long_df.head(3)

,지역별(1),STD_YYYYMM,뇌졸증
0,서울,2014,2012
1,부산,2014,391
2,대구,2014,673


In [81]:
long_df.rename(columns={'지역별(1)': 'SIDO'}, inplace=True)

In [82]:

# 변경할 문자열 매핑
name_mapping = {
    '서울': '서울특별시',
    '부산': '부산광역시',
    '대구': '대구광역시',
    '인천': '인천광역시',
    '광주': '광주광역시',
    '대전': '대전광역시',
    '울산': '울산광역시',
    '경기': '경기도',
    '세종': '세종특별자치시',
    '강원': '강원도',
    '충북': '충청북도',
    '충남': '충청남도',
    '전북': '전라북도',
    '전남': '전라남도',
    '경북': '경상북도',
    '경남': '경상남도',
    '제주': '제주특별자치도'
}

# 'SIDO' 컬럼의 값을 매핑에 따라 변경
long_df['SIDO'] = long_df['SIDO'].map(name_mapping)

In [83]:
# 'STD_YYYYMM' 열의 숫자가 아닌 문자를 제거
long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].str.replace('[^\d]', '', regex=True)

In [84]:
# 결과를 정수 타입으로 변환
long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].astype(int)

In [85]:
def increment_month_properly(date_int):
    date_str = str(date_int)
    # 년도만 있는 경우, 01월을 추가
    if len(date_str) == 4:
        return date_str + '01'
    # 년도와 한 자리 월이 있는 경우 (예: 20141)
    elif len(date_str) == 5:
        year, month = int(date_str[:4]), int(date_str[4:])
        new_month = month + 1 if month < 9 else 10  # 9월 이하면 1 증가, 그렇지 않으면 10월
        return f"{year}0{new_month}"
    # 년도와 두 자리 월이 있는 경우
    elif len(date_str) == 6:
        year, month = int(date_str[:4]), int(date_str[4:])
        # 월을 1 증가시키기 (12월인 경우 다음 해로 넘어가지 않음)
        new_month = month + 1 if month < 12 else month
        return f"{year}{new_month:02d}"
    else:
        return date_str  # 형식에 맞지 않는 경우 그대로 반환

long_df['STD_YYYYMM'] = long_df['STD_YYYYMM'].apply(increment_month_properly)


In [86]:
long_df['뇌졸증'].dtype

dtype('O')

In [87]:
# '뇌졸증' 컬럼을 수치형(float)으로 변환
long_df['뇌졸증'] = pd.to_numeric(long_df['뇌졸증'], errors='coerce')

# 누락된 값이 있는 행 찾기
missing_values_df = long_df[long_df['뇌졸증'].isna()]

# 'STD_YYYYMM' 별 '뇌졸증' 평균값 계산 및 소수점 첫째 자리까지 반올림
mean_values = long_df.groupby('STD_YYYYMM')['뇌졸증'].transform('mean').round(1)

# 누락된 '뇌졸증' 값에 평균값 할당
long_df['뇌졸증'] = long_df['뇌졸증'].fillna(mean_values)

In [88]:
long_df.to_csv('data/test/뇌졸증.csv', index=False)